In [8]:
## Loading the VGG16 classifier

from keras.applications import VGG16

img_rows = 224
img_cols = 224

vgg16 = VGG16(weights = 'imagenet',
             input_shape = (img_rows, img_cols, 3),
             include_top = False)

In [9]:
for (i, layer)  in enumerate(vgg16.layers):
    print(str(i) + " " + layer.__class__.__name__, layer.trainable)

0 InputLayer True
1 Conv2D True
2 Conv2D True
3 MaxPooling2D True
4 Conv2D True
5 Conv2D True
6 MaxPooling2D True
7 Conv2D True
8 Conv2D True
9 Conv2D True
10 MaxPooling2D True
11 Conv2D True
12 Conv2D True
13 Conv2D True
14 MaxPooling2D True
15 Conv2D True
16 Conv2D True
17 Conv2D True
18 MaxPooling2D True


In [13]:
from keras.applications import VGG16


## Let us work only with the top 4 layers

img_rows = 224
img_cols = 224

vgg16 = VGG16(weights = 'imagenet',
             input_shape = (img_rows, img_cols, 3),
             include_top = False)


## Let us make all the lower layers non trainable
for layer in vgg16.layers:
    layer.trainable = False
    
    
## Let us print our layers
for (i, layer)  in enumerate(vgg16.layers):
    print(str(i) + " " + layer.__class__.__name__, layer.trainable)

0 InputLayer False
1 Conv2D False
2 Conv2D False
3 MaxPooling2D False
4 Conv2D False
5 Conv2D False
6 MaxPooling2D False
7 Conv2D False
8 Conv2D False
9 Conv2D False
10 MaxPooling2D False
11 Conv2D False
12 Conv2D False
13 Conv2D False
14 MaxPooling2D False
15 Conv2D False
16 Conv2D False
17 Conv2D False
18 MaxPooling2D False


In [15]:
## Lets build a function that returns our Fully conneted layers head

def addTopLayer(bottomlayer, num_classes, D=256):
    top_model = bottomlayer.output
    top_model = Flatten(name = "flatten")(top_model)
    top_model = Dense(D, activation = "relu")(top_model)
    top_model = Dropout(0.3)(top_model)
    top_model = Dense(num_classes, activation = "softmax")(top_model)
    return top_model


In [20]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import MaxPooling2D, Conv2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model

num_classes = 17

FC_head = addTopLayer(vgg16, num_classes)

model =  Model(inputs = vgg16.input, outputs = FC_head)


print(model.summary())



Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [21]:
## Loading the dataset

from keras.preprocessing.image import ImageDataGenerator

train_data_dir = '17_flowers/train/'
validation_data_dir =  '17_flowers/validation/'


train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')


validation_datagen = ImageDataGenerator(rescale=1./255)


# Change the batchsize according to your system RAM
train_batchsize = 16
val_batchsize = 10


train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size = (img_rows, img_cols),
        batch_size = train_batchsize,
        class_mode = 'categorical')


validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size = (img_rows, img_cols),
        batch_size = val_batchsize,
        class_mode = 'categorical')

Found 1190 images belonging to 17 classes.
Found 170 images belonging to 17 classes.


In [24]:
## Training our top layers

from keras.optimizers import RMSprop
from keras.callbacks import EarlyStopping, ModelCheckpoint

checkpoint = ModelCheckpoint('best_vgg16_model.h5',
                            monitor = 'val_loss',
                            mode = 'min',
                            save_best_only = True,
                            verbose = 1)

early_stopping =  EarlyStopping(monitor = 'val_loss',
                               min_delta = 0,
                               patience = 3,
                               verbose = 1,
                               restore_best_weights = True)


callbacks = [checkpoint, early_stopping]


model.compile(loss = 'categorical_crossentropy',
             optimizer = RMSprop(lr = 0.001),
             metrics =  ['accuracy'])


nb_train_samples = 1190
nb_validation_samples = 170
epochs = 3
batch_size = 16

history  = model.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)

model.save("flowers_vgg.h5")

Epoch 1/3
74/74 [==============================] - ETA: 0s - loss: 2.2332 - accuracy: 0.2845
Epoch 00001: val_loss improved from inf to 1.17229, saving model to best_vgg16_model.h5
74/74 [==============================] - 124s 2s/step - loss: 2.2332 - accuracy: 0.2845 - val_loss: 1.1723 - val_accuracy: 0.6900
Epoch 2/3
74/74 [==============================] - ETA: 0s - loss: 1.8164 - accuracy: 0.4250
Epoch 00002: val_loss improved from 1.17229 to 1.07065, saving model to best_vgg16_model.h5
74/74 [==============================] - 128s 2s/step - loss: 1.8164 - accuracy: 0.4250 - val_loss: 1.0706 - val_accuracy: 0.6700
Epoch 3/3
74/74 [==============================] - ETA: 0s - loss: 1.5125 - accuracy: 0.5043
Epoch 00003: val_loss improved from 1.07065 to 0.84675, saving model to best_vgg16_model.h5
74/74 [==============================] - 130s 2s/step - loss: 1.5125 - accuracy: 0.5043 - val_loss: 0.8468 - val_accuracy: 0.7700
